<a href="https://colab.research.google.com/github/davide1536/minesweeper/blob/main/mineSweeper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install python-sat[pblib,aiger]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [107]:
import random 
import numpy as np
from pysat.solvers import Solver
from pysat.card import *

In [30]:
def intersection(lst1, lst2):
  return list(set(lst1).intersection(lst2))

In [31]:
def difference(list1, list2):
  return list(set(list1) - set(list2))

In [99]:
def selectCell(board, knowledge, tried):
  for cells in knowledge.totalCells:
    for cell in cells:
      if board[cell].safe != 1 and board[cell].flag != 1 and not(cell in tried):
        return cell

In [82]:
def createClauses(board, knowledge):
  clauses = []
  for i,cells in enumerate(knowledge.totalCells):
    if knowledge.count[i] != 0 and knowledge.count[i] != len(cells):
      cnfClause = CardEnc.equals(lits = cells, bound = int(knowledge.count[i]))
      clauses.append(cnfClause)

  return clauses


In [61]:
def createAssumptions(board,knowledge):
  assumptions = []
  for i, cells in enumerate(knowledge.totalCells):
    if knowledge.count[i] == 0:
      for cell in cells:
        assumptions.append(board[cell].proposition)
    elif knowledge.count[i] == len(cells):
      for cell in cells:
        assumptions.append(board[cell].proposition)

  return assumptions

In [32]:
def removePositiveProposition(knowledge, currentCells, currentIndex, dictionary):
  for i,cells in enumerate(knowledge.totalCells):
    if i != currentIndex:
      removeAndUpdateProposition(knowledge, currentCells, cells, dictionary, i)
      #print("differenza: ", knowledge.totalCells[i])
      if knowledge.totalCells[i] == []:
        #print("tolgo liste vuote")
        del knowledge.totalCells[i]
        del knowledge.count[i]

In [33]:
def removeNegativeProposition(knowledge, currentCells, currentIndex, dictionary):
  for i,cells in enumerate(knowledge.totalCells):
    if i != currentIndex:
      numberOfCells = removeAndUpdateProposition(knowledge, currentCells, cells, dictionary,i)
      #print("differenza: ", knowledge.totalCells[i])
      #print("counter attuale: ",knowledge.count[i])
      knowledge.count[i] -= numberOfCells
      #print("counter aggiornato: ", knowledge.count[i])
      if knowledge.totalCells[i] == []:
       # print("tolgo lister vuote")
        del knowledge.totalCells[i]
        del knowledge.count[i]

In [34]:
def removeAndUpdateProposition(knowledge, currentCells, cells, dictionary, index):

  #print("rimuovo proposizioni")
  intersect= intersection(cells, currentCells)
  #print("intersezione tra ", cells, " e ",currentCells, " é ", intersect)
  diff = difference(knowledge.totalCells[index], intersect)
  knowledge.totalCells[index] = diff
  return len(intersect)
 

In [87]:
def updateKnowledge(knowledge,board):
  for i,cells in enumerate(knowledge.totalCells):
      if knowledge.count[i] == 0:
        #print("trovato celle con counter 0")
        for cell in cells:
          board[cell].safe = 1
        removePositiveProposition(knowledge, cells, i, board)
      if knowledge.count[i] == len(cells):
        #print("trovato celle non safe")
        for cell in cells:
          board[cell].safe = 0
          board[cell].flag = 1
          board[cell].proposition = - board[cell].proposition
        removeNegativeProposition(knowledge, cells, i,board)

In [36]:
def buildCellDictionary(board):
  dictionary = {}
  for row in range(SIZE):
    for col in range(SIZE):
      dictionary[codeId(row, col)] = createCell(board[row,col], row, col)
  
  return dictionary


In [37]:
def codeId(value1, value2):
  return value1 * SIZE + value2

In [121]:
def findSafeCell(board, knowledge):
 
  cellTried = []
  assumptions = createAssumptions(board, knowledge)
  clauses = createClauses(board, knowledge)
  
  cell = selectCell(board,knowledge, cellTried)
  cellTried.append(cell)
  
  print("provo a selezionare la cella: ", cell)

  positiveSolver = Solver()
  for formulas in clauses:
    print("clauses: ", formulas.clauses)
  positiveSolver.append_formula(formula=clauses)
  print("proposition: ", int(-board[cell].proposition))
  positiveSolver.add_clause([int(-board[cell].proposition)])

  negativeSolver = Solver()
  negativeSolver.append_formula(clauses, no_return=True)
  negativeSolver.add_clause([int(board[cell].proposition)])

  solution = positiveSolver.solve(assumptions=assumptions)
  if solution == False:
    print("ho trovato una cella safe")
    return cell
  else:
    print("ho trovato una mina")
    solution = negativeSolver.solve(assumptions = assumptions)
    if solution == False:
      createKnowledge(board[cell], board, knowledge)


In [93]:
def createKnowledge(cell, board, knowledge):
  #print("creo knowledge della cella in posizione: ", cell.row, " ", cell.col)

  currentKnowledge = []
  if cell.value == -1:
    currentKnowledge.append(cell.id)
    knowledge.count.append(1)
  else:
    
    value = cell.value
    for i in range(3):
      for j in range(3):
        if (cell.row-1+i >= 0 and cell.row-1+i <= SIZE-1) and (cell.col-1+j >= 0 and cell.col-1+j <= SIZE-1):
          if cell.id != board[codeId(cell.row-1+i,cell.col-1+j)].id:
              currentKnowledge.append(board[codeId(cell.row-1+i,cell.col-1+j)].id)
  
    knowledge.count.append(cell.value)

  knowledge.totalCells.append(currentKnowledge)
  print("knowledge accumulata sino ad ora")
  for i,cells in enumerate(knowledge.totalCells):
    print(cells, " ", knowledge.count[i])
  updateKnowledge(knowledge,board)

  print("knowledge aggiornata")
  for i,cells in enumerate(knowledge.totalCells):
    print(cells, " ", knowledge.count[i])
  # print("dopo l'aggiornamento: ")
  # for i,cells in enumerate(knowledge.totalCells):
  #   print(cells, " ", knowledge.count[i])


In [40]:
def checkTrivialSafeCell(board, knowledge):
  for i,cells in enumerate(knowledge.totalCells):
    if knowledge.count[i] == 0:
      for cell in cells:
        if board[cell].selected == 0:
          return cell
  
  return -1

In [55]:
def selectNextSafeCell(board, knowledge):
  nextCell = checkTrivialSafeCell(board, knowledge)
  #if there aren't any trivial safe cell use sat solver on clause
  if nextCell == -1:
    nextCell = findSafeCell(board, knowledge)
  
  if nextCell == -1:
    result = nextCell
  else:
    result = board[nextCell]
    
  return result


In [42]:
def countMines(row,col,board, size):
  counter = 0

  for i in range(3):
    for j in range(3):
      if (row-1+i >= 0 and row-1+i <= size-1) and (col-1+j >= 0 and col-1+j <= size-1):
        if board[row-1+i, col-1+j] == -1:
          counter += 1
        
     
  
  return counter

In [43]:
#function that, for each cell, counts the number of mines around it
def fillBoard(board, size):
  
  for row in range(size):
    for col in range(size):
      if board[row,col] != -1:
        n_mines = countMines(row, col, board, size)
        board[row,col] = n_mines
    
      
    
  return board


In [44]:

def buildBoard(size, probability):
  total_mines = 0
  #board[i][j] = 0 --> no bomb
  #board[i][j] = 1 --> bomb
  print("inizializzo board di gioco")
  board = np.zeros((size,size))
  for row in range(size):
    for col in range(size):
      randomNumber = random.uniform(0,1)
      if randomNumber <= probability:
        board[row,col] = -1
        total_mines += 1
      else:
        board[row,col] = 0


  board = fillBoard(board, size)
  print("board di gioco inizializzata con ",total_mines, " ","mine")
  return board
      

In [117]:
def createCell(value, row, col):
  cell = Cell()
  cell.value = int(value)
  if cell.value >= 0:
    cell.safe = 1
  else:
    cell.safe = 0
    
  cell.selected = 0
  cell.flag = 0
  cell.row = row
  cell.col = col
  cell.proposition = int(codeId(row,col))
  cell.id = cell.proposition
  return cell

In [46]:
def startGame(board, knowledge):
  result = ""
  firstCell = board[codeId(0,0)]
  firstCell.selected = 1

  if firstCell.value == -1:
    result = "lose"
  else:
    createKnowledge(firstCell, board, knowledge)

 

  while not(result == "win") and not(result == "lose"):
    
    cell = selectNextSafeCell(board, knowledge)
    if cell == -1:
      return "could not find a safe cell"

    else:
      print("ho selezionato la cella ", cell.id)
      cell.selected = 1
    
      if cell.value == -1:
        result = "lose"
      else:
        createKnowledge(cell, board, knowledge)


  return result

In [47]:
class InternalRepresentation:
  def __init__(self) -> None:
    self.totalCells = []
    self.count = []

In [85]:
class Cell:
  def __init__(self) -> None:
    self.safe = 0
    self.value = 0
    self.proposition = 0
    self.id = 0
    self.selected = 0
    self.row = 0
    self.col = 0
    self.flag = 0

In [73]:
#build a 9x9 board
SIZE = 9
bomb_probability = 1/6
board = buildBoard(SIZE, bomb_probability)
print(board)

inizializzo board di gioco
board di gioco inizializzata con  13   mine
[[ 0.  0.  1.  1.  1.  0.  0.  1. -1.]
 [ 1.  2.  2. -1.  1.  0.  0.  1.  1.]
 [-1.  3. -1.  2.  1.  0.  0.  0.  0.]
 [-1.  3.  2.  2.  2.  1.  1.  1.  1.]
 [ 1.  1.  2. -1.  3. -1.  1.  1. -1.]
 [ 0.  0.  2. -1.  3.  2.  2.  2.  1.]
 [ 1.  2.  3.  2.  1.  1. -1.  2.  1.]
 [ 1. -1. -1.  1.  0.  1.  1.  2. -1.]
 [ 1.  2.  2.  1.  0.  0.  0.  1.  1.]]


In [122]:
#start game from the up-left corner
knowledge = InternalRepresentation();
boardDictionary = buildCellDictionary(board)
result = startGame(boardDictionary, knowledge)
print("risultato: ", result);      

knowledge accumulata sino ad ora
[1, 9, 10]   0
knowledge aggiornata
[1, 9, 10]   0
ho selezionato la cella  1
knowledge accumulata sino ad ora
[1, 9, 10]   0
[0, 2, 9, 10, 11]   0
knowledge aggiornata
[1, 10, 9]   0
[0, 2, 11]   0
ho selezionato la cella  10
knowledge accumulata sino ad ora
[1, 10, 9]   0
[0, 2, 11]   0
[0, 1, 2, 9, 11, 18, 19, 20]   2
knowledge aggiornata
[1, 10, 9]   0
[0, 2, 11]   0
[18, 19, 20]   2
ho selezionato la cella  9
knowledge accumulata sino ad ora
[1, 10, 9]   0
[0, 2, 11]   0
[18, 19, 20]   2
[0, 1, 10, 18, 19]   1
knowledge aggiornata
[1, 10, 9]   0
[0, 2, 11]   0
[18, 19, 20]   2
[18, 19]   1
ho selezionato la cella  2
knowledge accumulata sino ad ora
[1, 10, 9]   0
[0, 2, 11]   0
[18, 19, 20]   2
[18, 19]   1
[1, 3, 10, 11, 12]   1
knowledge aggiornata
[1, 10, 9]   0
[0, 2, 11]   0
[18, 19, 20]   2
[18, 19]   1
[3, 12]   1
ho selezionato la cella  11
knowledge accumulata sino ad ora
[1, 10, 9]   0
[0, 2, 11]   0
[18, 19, 20]   2
[18, 19]   1
[3, 12] 

TypeError: ignored